In [1]:
import os
from dotenv import load_dotenv

# 필요한 라이브러리 import
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# 환경 변수 로드 및 검증
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다.")


In [ ]:
# PDF 문서 로드
print("PDF 문서를 로드하는 중...")
loader = PyPDFLoader("data/tax_with_table.pdf")
pages = loader.load()
print(f"총 {len(pages)}개의 페이지가 로드되었습니다.")

# 텍스트 분할 (청크 크기 및 중복 크기 최적화)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  
    chunk_overlap=50,  
    separators=["\n\n", "\n", ".", " ", ""]  # 마침표 구분자 추가
)
splits = text_splitter.split_documents(pages)
print(f"총 {len(splits)}개의 청크로 문서가 분할되었습니다.")


PDF 문서를 로드하는 중...
총 84개의 페이지가 로드되었습니다.
총 164개의 청크로 문서가 분할되었습니다.


In [9]:
# 임베딩 모델 설정
embeddings_model = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model="text-embedding-3-small"
)

# 벡터 저장소 생성 - 배치 처리
print("\n벡터 저장소를 생성하는 중...")
batch_size = 50

try:
    first_batch = splits[:batch_size]
    print(f"첫 번째 배치 처리 중... ({len(first_batch)}개 문서)")
    db = Chroma.from_documents(
        first_batch, 
        embeddings_model, 
        persist_directory="./chroma_db"
    )
    print("첫 번째 배치 처리 완료")
    
    # 나머지 배치들을 순차적으로 추가
    for i in range(batch_size, len(splits), batch_size):
        batch = splits[i:i + batch_size]
        batch_num = i // batch_size + 1
        total_batches = len(splits) // batch_size + (1 if len(splits) % batch_size else 0)
        
        print(f"배치 {batch_num}/{total_batches} 처리 중... ({len(batch)}개 문서)")
        db.add_documents(batch)
        print(f"배치 {batch_num} 처리 완료")

except Exception as e:
    print(f"벡터 저장소 생성 중 오류: {str(e)}")
    exit(1)

print("모든 문서의 벡터화가 완료되었습니다!")


벡터 저장소를 생성하는 중...
첫 번째 배치 처리 중... (50개 문서)
첫 번째 배치 처리 완료
배치 2/4 처리 중... (50개 문서)
배치 2 처리 완료
배치 3/4 처리 중... (50개 문서)
배치 3 처리 완료
배치 4/4 처리 중... (14개 문서)
배치 4 처리 완료
모든 문서의 벡터화가 완료되었습니다!


In [16]:
# 질문 설정
query = "비과세소득에 해당하는 소득은 어떤 것들이 있나요? 비과세소득에 대하여 자세히 설명해 주세요."
print(f"\n=== 검색 성능 테스트 ===")
print(f"질문: {query}")

# 1. 다양한 검색 방법으로 테스트
print("\n1. 기본 유사도 검색 (상위 10개):")
docs_basic = db.similarity_search(query, k=10)
for i, doc in enumerate(docs_basic):
    print(f"  {i+1}. {doc.page_content[:100]}...")

# 2. 유사도 점수와 함께 검색하여 성능 확인
print("\n2. 유사도 점수 확인:")
docs_with_scores = db.similarity_search_with_score(query, k=10)
print("점수 분포:")
for i, (doc, score) in enumerate(docs_with_scores):
    print(f"  순위 {i+1}: 점수 {score:.4f}")
    if i < 3:  # 상위 3개만 내용 표시
        print(f"    내용: {doc.page_content[:150]}...")

# 최고 점수와 최저 점수 확인
if docs_with_scores:
    best_score = docs_with_scores[0][1]
    worst_score = docs_with_scores[-1][1]
    print(f"\n점수 범위: {best_score:.4f} ~ {worst_score:.4f}")
    
    # 적절한 임계값 제안
    suggested_threshold = best_score * 0.7  # 최고 점수의 70%
    print(f"제안 임계값: {suggested_threshold:.4f}")


=== 검색 성능 테스트 ===
질문: 비과세소득에 해당하는 소득은 어떤 것들이 있나요? 비과세소득에 대하여 자세히 설명해 주세요.

1. 기본 유사도 검색 (상위 10개):
  1. 소득세법 
법제처  3 
 국가법령정보센터 
제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) ① 신탁재산에 귀속되는 소득은 그 신탁의 이익을 받을 
수익자(수익자가 사망하는...
  2. 소득세법 
법제처  3 
 국가법령정보센터 
제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) ① 신탁재산에 귀속되는 소득은 그 신탁의 이익을 받을 
수익자(수익자가 사망하는...
  3. 소득세법 
법제처  7 
 국가법령정보센터 
제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25...
  4. 소득세법 
법제처  7 
 국가법령정보센터 
제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25...
  5. 소득세법 
법제처  10 
 국가법령정보센터 
등이 받는 수당 
[전문개정 2009. 12. 31.] 
 
제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아...
  6. 소득세법 
법제처  10 
 국가법령정보센터 
등이 받는 수당 
[전문개정 2009. 12. 31.] 
 
제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아...
  7. 소득세법 
법제처  30 
 국가법령정보센터 
과세기간의 소득금액을 계산할 때 총수입금액에 산입하지 아니한다. 
⑦ 개별소비세 및 주세의 납세의무자인 거주자가 자기의 총수입금액으로...
  8. 소득세법 
법제처  30 
 국가법령정보센터 
과세기간의 소득금액을 계산할 때 총수입금액에 산입하지 아니한다. 
⑦ 개별소비세 및 주세의 납세의무자인 거주자가 자기의 총수입금액으로...
  9. 소득세법 
법제처  28 
 국가법령정보센터

In [17]:
# 3. 다양한 retriever 설정으로 테스트
retriever_configs = [
    {
        "name": "기본 설정 (상위 4개)",
        "search_type": "similarity",
        "search_kwargs": {"k": 4}
    },
    {
        "name": "더 많은 문서 (상위 8개)",
        "search_type": "similarity", 
        "search_kwargs": {"k": 8}
    },
    {
        "name": "임계값 적용 (0.3)",
        "search_type": "similarity_score_threshold",
        "search_kwargs": {"score_threshold": 0.3, "k": 10}
    },
    {
        "name": "낮은 임계값 (0.1)",
        "search_type": "similarity_score_threshold",
        "search_kwargs": {"score_threshold": 0.1, "k": 10}
    }
]

print("\n3. 다양한 retriever 설정 테스트:")
best_config = None
best_response = None

for config in retriever_configs:
    print(f"\n--- {config['name']} ---")
    
    try:
        retriever = db.as_retriever(
            search_type=config["search_type"],
            search_kwargs=config["search_kwargs"]
        )
        
        # 간단한 프롬프트로 테스트
        template = '''다음 문맥을 바탕으로 질문에 답변해주세요:
        <문맥>
        {context}
        </문맥>

        질문: {input}

        답변:'''
        
        prompt = ChatPromptTemplate.from_template(template)
        
        model = ChatOpenAI(
            model='gpt-3.5-turbo',
            temperature=0,
            api_key=OPENAI_API_KEY
        )
        
        document_chain = create_stuff_documents_chain(model, prompt)
        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        
        response = retrieval_chain.invoke({"input": query})
        
        print(f"검색된 문서 수: {len(response.get('context', []))}")
        print(f"답변 길이: {len(response['answer'])}")
        print(f"답변 미리보기: {response['answer'][:200]}...")
        
        # "찾을 수 없습니다"가 없고 실질적인 답변이 있는 경우를 우선시
        if "찾을 수 없습니다" not in response['answer'] and len(response['answer']) > 50:
            if best_config is None or len(response['answer']) > len(best_response['answer']):
                best_config = config
                best_response = response
                print("현재 최적 설정!")
                
    except Exception as e:
        print(f" 오류 발생: {str(e)}")


3. 다양한 retriever 설정 테스트:

--- 기본 설정 (상위 4개) ---
검색된 문서 수: 4
답변 길이: 483
답변 미리보기: 비과세소득에 해당하는 소득은 다음과 같습니다:
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득
   - 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득
   - 1 개의 주택을 소유하는 자의 주택임대소득
   - 대통령령으로 정하는 농어가부업소득
   - 대통령령으로 정하는 전통주의 제조에서 발생하...
현재 최적 설정!

--- 더 많은 문서 (상위 8개) ---
검색된 문서 수: 8
답변 길이: 426
답변 미리보기: 비과세소득에 해당하는 소득은 다음과 같습니다:
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득, 1 개의 주택을 소유하는 자의 주택임대소득, 대통령령으로 정하는 농어가부업소득, 대통령령으로 정하는 전통주의 제조에서 발생하는 소득, 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는...

--- 임계값 적용 (0.3) ---


No relevant docs were retrieved using the relevance score threshold 0.3


검색된 문서 수: 0
답변 길이: 274
답변 미리보기: 비과세소득에 해당하는 소득은 일반적으로 세금이 부과되지 않는 소득을 말합니다. 이는 국가의 세법에 따라 다를 수 있지만, 대부분의 경우 특정 조건을 충족하는 소득이 해당됩니다. 예를 들어, 주택임대소득, 특정 금융상품의 이자소득, 특정 보험금 등이 비과세소득에 해당할 수 있습니다. 이러한 소득은 세금이 부과되지 않기 때문에 세금을 납부할 필요가 없습니다. ...

--- 낮은 임계값 (0.1) ---
검색된 문서 수: 10
답변 길이: 413
답변 미리보기: 비과세소득에 해당하는 소득은 다음과 같습니다:
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득, 1 개의 주택을 소유하는 자의 주택임대소득, 대통령령으로 정하는 농어가부업소득, 대통령령으로 정하는 전통주의 제조에서 발생하는 소득, 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는...


In [18]:
# 최적 설정으로 최종 답변 생성
print(f"\n=== 최종 답변 (최적 설정: {best_config['name'] if best_config else '기본 설정'}) ===")

if best_config:
    final_retriever = db.as_retriever(
        search_type=best_config["search_type"],
        search_kwargs=best_config["search_kwargs"]
    )
else:
    # 기본 설정 사용
    final_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 8}
    )

# 최종 프롬프트 (더 관대한 조건)
final_template = '''다음 문맥을 참고하여 질문에 답변해주세요. 
문맥에서 직접적인 답변을 찾을 수 없더라도, 관련된 정보가 있다면 그것을 바탕으로 답변해주세요.

<문맥>
{context}
</문맥>

질문: {input}

답변:'''

final_prompt = ChatPromptTemplate.from_template(final_template)
final_document_chain = create_stuff_documents_chain(model, final_prompt)
final_retrieval_chain = create_retrieval_chain(final_retriever, final_document_chain)

final_response = final_retrieval_chain.invoke({"input": query})

print("\n=== 최종 답변 ===")
print(final_response["answer"])

print(f"\n=== 참조된 문서 ({len(final_response.get('context', []))}개) ===")
if final_response.get("context"):
    for i, doc in enumerate(final_response["context"], 1):
        print(f"\n{i}. 문서 내용:")
        print(doc.page_content[:300] + "...")
else:
    print("참조된 문서가 없습니다.")

print("\n 처리가 완료되었습니다.")


=== 최종 답변 (최적 설정: 기본 설정 (상위 4개)) ===

=== 최종 답변 ===
비과세소득에 해당하는 소득은 다음과 같습니다.
1. 「공익신탁법」에 따른 공익신탁의 이익
2. 사업소득 중 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득, 1 개의 주택을 소유하는 자의 주택임대소득(단, 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외), 대통령령으로 정하는 농어가부업소득, 대통령령으로 정하는 전통주의 제조에서 발생하는 소득, 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득(연 600만원 이하의 금액), 대통령령으로 정하는 작물재배업에서 발생하는 소득, 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득
3. 근로소득과 퇴직소득 중 특정 조건을 충족하는 소득

비과세소득은 소득세를 과세하지 않는 소득으로, 이에 해당하는 소득은 세금을 납부하지 않아도 되는 소득을 말합니다. 이는 해당 소득이 특정 목적을 위해 발생하거나 특정 조건을 충족하는 경우에 해당하게 됩니다.

=== 참조된 문서 (4개) ===

1. 문서 내용:
소득세법 
법제처  3 
 국가법령정보센터 
제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) ① 신탁재산에 귀속되는 소득은 그 신탁의 이익을 받을 
수익자(수익자가 사망하는 경우에는 그 상속인)에게 귀속되는 것으로 본다. 
② 제1항에도 불구하고 위탁자가 신탁재산을 실질적으로 통제하는 등 대통령령으로 정하는 요건을 충족하는 
신탁의 경우에는 그 신탁재산에 귀속되는 소득은 위탁자에게 귀속되는 것으로 본다.<개정 2023. 12. 31.> 
[본조신설 2020. 12. 29.] 
 
제3조(과세소득의 범위) ① 거주자에게는 이 ...

2. 문서 내용:
소득세법 
법제처  3 
 국가법령정보센터 
제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) ① 신탁재산에 귀속되는 소득은 그 신탁의 이익을 받을 
수익자(수익자가 사망하는 경우에는 그 상속인)에